In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
import pandas as pd
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df.head()

In [33]:
df.isnull().sum()

In [34]:
import numpy as np
df['Outcome'] = np.where(df['Outcome']==1, "Diabetic", "Not diabetic")

In [35]:
import seaborn as sns
sns.pairplot(df, hue='Outcome')

In [36]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [37]:
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

In [82]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [86]:
np.shape(X_test)

In [84]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [85]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [72]:
df.shape

# Model 1 

In [154]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features=2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    def forward(self, x):
        x = F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = self.out(x)
        return x

# Model 2

In [123]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=16, hidden2=32,hidden3=64,hidden4=32, hidden5=8, out_features=2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.f_connected3 = nn.Linear(hidden2, hidden3)
        self.f_connected4 = nn.Linear(hidden3, hidden4)
        self.f_connected5 = nn.Linear(hidden4, hidden5)
        self.out = nn.Linear(hidden5, out_features)
    def forward(self, x):
        x = F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = F.relu(self.f_connected3(x))
        x = F.relu(self.f_connected4(x))
        x = F.relu(self.f_connected5(x))
        x = self.out(x)
        return x

# Model 3

In [124]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=16, hidden2=32,hidden3=16,hidden4=8, out_features=2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.f_connected3 = nn.Linear(hidden2, hidden3)
        self.f_connected4 = nn.Linear(hidden3, hidden4)
        self.out = nn.Linear(hidden4, out_features)
    def forward(self, x):
        x = F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = F.relu(self.f_connected3(x))
        x = F.relu(self.f_connected4(x))
        x = self.out(x)
        return x

In [155]:
torch.manual_seed(20)
model = ANN_Model()

In [156]:
model.parameters

In [157]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [158]:
epochs = 500
final_losses = []
for i in range(epochs):
    i=i+1
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_losses.append(loss)
    if i%10==0:
        print("Epoch number: {} and the loss: {}".format(i,loss))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [159]:
import matplotlib.pyplot as plt
%matplotlib inline

with torch.no_grad():
    plt.plot(range(epochs), final_losses)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')

In [160]:
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

In [161]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm

In [162]:
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [91]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)
score

In [92]:
torch.save(model,'diabetes.pt')

In [93]:
model=torch.load('diabetes.pt')

In [94]:
model.eval()

In [95]:
list(df.iloc[0,:-1])

In [96]:
lst1=[6.0, 130.0, 72.0, 40.0, 0.0, 25.6, 0.627, 45.0]

In [97]:
new_data=torch.tensor(lst1)

In [98]:
with torch.no_grad():
    print(model(new_data))
    print(model(new_data).argmax().item())

In [ ]:
y = df